In [4]:
# compute and save spatiotemporal derivatives

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os, cv2
from scipy import io
from scipy.signal import sepfir2d, medfilt
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata

In [2]:
src = '../publicdataset/images/'
dest = './FSderivs'
calib_path = '../CameraCalibration/camera_data/'
dates = [f'Oct1{i}' for i in range(2,10) if i!=8]
speeds = ['Wy_50','Wy_20']
os.listdir(src)

['Vz_60',
 'Vx_60',
 'Vxz_60',
 'Wy_20',
 'Wy_50',
 'Vz_40',
 'Vx_40',
 'Vxz_40',
 'Vz_40+Wy_45',
 'Vz_60+Wy_50',
 'Vz_40+Wy_15',
 'Vz_60+Wy_15']

In [21]:
# file management

insta1dates = ['Oct16','Oct17','Oct18','Oct19']
insta2dates = ['Oct12','Oct13','Oct14','Oct15']

def getinsta(filename):
    insta = -1
    for date in insta1dates:
        if date in filename:
            insta = 1
    for date in insta2dates:
        if date in filename:
            if insta == 1:
                print(f'error in getinsta on {filename}')
            insta = 2
    return insta

def listpngs(vid):
    rawpngs = os.listdir(vid)
    pngnums = sorted([int(png[:-4]) for png in rawpngs])
    pngs = [os.path.join(vid,str(pngnum)+'.png') for pngnum in pngnums]
    return pngs

def getpngs(speed,date,sec):
    viddir = f'{pngs_path}{speed}/{date}'
    if  not  os.path.exists(viddir):
        return []
    pngs = listpngs(viddir)
    if len(pngs) < sec*fps:
        return pngs
    else:
        first = sec*fps
        last = min(len(pngs),(sec+1)*fps+2*timestep+1) #updated from (sec+1)*fps+timestep before cleanup 
        return pngs[first:last]

In [22]:
# image reprojection and conversion from pixel values to luminance

pts3d = np.zeros([1504*1504,3,3,2])
for insta in [1,2]:
    for cam in [0,1]:
        calibdata = io.loadmat(os.path.join(calib_path,f'insta{insta}_spherical_coordinates_water_xyz.mat'))
        pts3d[:,0,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['x'][0][0])
        pts3d[:,1,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['y'][0][0])
        pts3d[:,2,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['z'][0][0])

def Rx(gamma):
    return [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]

def Ry(beta):
    return [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]

def Rz(alpha):
    return [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]

def makeR(yaw,pitch=0,roll=0):
    return np.matmul(np.matmul(Rx(pitch),Ry(yaw)),Rz(roll))

colorchannel = 1
sig1 = 3
sig2 = 2
case = plt.imread(f'{calib_path}dive_case_image.png')[:,:,1]
case = (case>0.5).astype('uint8')

LUT01 = io.loadmat(os.path.join('calibration_data/pixel_value_LUT_insta1_basic.mat'))
LUT02 = io.loadmat(os.path.join('calibration_data/pixel_value_LUT_insta2_basic.mat'))
LUT1 = np.stack([LUT01['B_response_LUT'][0],LUT01['G_response_LUT'][0],LUT01['R_response_LUT'][0]],axis=-1)
LUT2 = np.stack([LUT02['B_response_LUT'][0],LUT02['G_response_LUT'][0],LUT02['R_response_LUT'][0]],axis=-1)

def lummap(im,insta):
    if insta == 1:
        lum = np.stack([cv2.LUT((255*im[:,:,i]).astype('uint8'),LUT1[:,i]) for i in range(3)],axis=-1).astype('float32')
    else:
        lum = np.stack([cv2.LUT((255*im[:,:,i]).astype('uint8'),LUT2[:,i]) for i in range(3)],axis=-1).astype('float32')
    return lum

def tanlumfromfile(filename,pitch=0,yaw=0,FOV=40,res=40,foclen=1.0,colorchannel=colorchannel,lum=False):
    insta = getinsta(filename)
    im = cv2.imread(filename)
    if lum:
        im = lummap(im,insta)
    im = im[:,:,colorchannel]*case
    if np.abs(yaw) < np.pi/2:
        cam = 0
        im = gaussian_filter(np.rot90(im[:,:1504],3),sig1)
    else:
        cam = 1
        yaw = (yaw-np.pi)%np.pi
        im = gaussian_filter(np.rot90(im[:,1504:]),sig2)
    R = makeR(yaw,pitch)
    rotpts3d = np.matmul(pts3d[:,:,insta,cam],R)
    X = rotpts3d[:,0].reshape([1504,1504])
    Y = rotpts3d[:,1].reshape([1504,1504])
    Z = rotpts3d[:,2].reshape([1504,1504])
    x = X*foclen/(Z+1e-10)
    y = Y*foclen/(Z+1e-10)
    lim = np.tan(FOV/2*np.pi/180)
    clip = ~np.logical_or(np.logical_or(x>lim,x<-lim),np.logical_or(y>lim,y<-lim))
    pixx,pixy = np.meshgrid(np.linspace(-lim,lim,res),np.linspace(-lim,lim,res))
    interp = griddata((x[clip],y[clip]),im[clip],(pixx,pixy),'cubic')
    return interp

In [23]:
# sampling parameters
FOV = 40
res = 40
taps = 9
maxframes = 400
pfilt = [0.000721, 0.015486, 0.090341, 0.234494, 0.317916, 0.234494, 0.090341, 0.015486, 0.000721] # prefilter
dfilt = [0.003059, 0.035187, 0.118739, 0.143928, 0.000000, -0.143928, -0.118739, -0.035187, -0.003059] # derivative
yaws = [0,np.pi]
pitchesdeg = np.linspace(40,-60,11)
pitches = np.multiply(np.pi/180,pitchesdeg)

In [106]:
def saveFSderivs(speed,date):
    if not os.path.exists(dest+speed+'/'):
        os.makedirs(dest+speed+'/')
    os.makedirs(meddest+speed+'/',exist_ok=True)
    
    viddir = src+speed+'/'+date
    pngs = listpngs(viddir)
    if len(pngs) < 100:
        print(f'\t\terror in {viddir}')
        return
    
    I = np.nan*np.ones([taps,res,res]) # active image block
    
    Ix = np.nan*np.ones([len(yaws),len(pitches),len(pngs)-taps,res,res])
    Iy = np.nan*np.ones_like(Ix)
    It = np.nan*np.ones_like(Ix)
     
    for y in range(len(yaws)):
        for p in range(len(pitches)):
            print(f'\n\t\t{y}, {pitchesdeg[p]}: ',end='')
            
            # initiate image block
            for i in range(taps):
                I[i] = tanlumfromfile(pngs[i],pitch=pitches[p],yaw=yaws[y],FOV=FOV,res=res+2,foclen=1.0,
                                      lum=True,colorchannel=1)[1:-1,1:-1]
                
            if np.any(np.isnan(I)):
                for i in range(taps):
                    print(f'{np.sum(np.isnan(I[i]))} nans at init',flush=True)    

            for i in range(taps,min(maxframes,len(pngs)-1)):
                
                # update image block
                I = np.roll(I,-1,0)
                I[-1] = tanlumfromfile(pngs[i],pitch=pitches[p],yaw=yaws[y],
                                       FOV=FOV,res=res+2,foclen=1.0,
                                       lum=True,colorchannel=1)[1:-1,1:-1]
                                
                
                # take derivatives
                fdt = np.zeros([res,res])
                fpt = np.zeros([res])
                for k in range(taps):
                    fdt = fdt + dfilt[taps-k-1]*I[k] # differentiate in time
                    fpt = fpt + pfilt[taps-k-1]*I[k] # prefilter in time

                    
                Ix[y,p,i-taps]  = sepfir2d( fpt, dfilt, pfilt ) # spatial (x) derivative
                Iy[y,p,i-taps]  = sepfir2d( fpt, pfilt, dfilt ) # spatial (y) derivative
                It[y,p,i-taps]  = sepfir2d( fdt, pfilt, pfilt ) # temporal derivative
                
                if np.any(np.isnan(Ix[y,p,i-taps])):
                    print(f'{np.sum(np.isnan(Ix[y,p,i-taps]))} Ix nans at {y,p,i}',flush=True)
               
                # intermediate save
                if not i%50:
                    print(i,end=' ')
                    np.savez(dest+speed+'/'+date,Ix=Ix,Iy=Iy,It=It)
                  
    # final save
    print(i)
    np.savez(dest+speed+'/'+date,Ix=Ix,Iy=Iy,It=It)

In [108]:
for speed in ['Wy_20']:
    print(speed)
    for date in dates[2:]:
        print('\t'+date)
        saveFSderivs(speed,date)

Wy_20
	Oct14

		0, 40.0: 50 100 150 200 250 300 350 
		0, 30.0: 50 100 150 200 250 300 350 
		0, 20.0: 50 100 150 200 250 300 350 
		0, 10.0: 50 100 150 200 250 300 350 
		0, 0.0: 50 100 150 200 250 300 350 
		0, -10.0: 50 100 150 200 250 300 350 
		0, -20.0: 50 100 150 200 250 300 350 
		0, -30.0: 50 100 150 200 250 300 350 
		0, -40.0: 50 100 150 200 250 300 350 
		0, -50.0: 50 100 150 200 250 300 350 
		0, -60.0: 50 100 150 200 250 300 350 
		1, 40.0: 50 100 150 200 250 300 350 
		1, 30.0: 50 100 150 200 250 300 350 
		1, 20.0: 50 100 150 200 250 300 350 
		1, 10.0: 50 100 150 200 250 300 350 
		1, 0.0: 50 100 150 200 250 300 350 
		1, -10.0: 50 100 150 200 250 300 350 
		1, -20.0: 50 100 150 200 250 300 350 
		1, -30.0: 50 100 150 200 250 300 350 
		1, -40.0: 50 100 150 200 250 300 350 
		1, -50.0: 50 100 150 200 250 300 350 
		1, -60.0: 50 100 150 200 250 300 350 398
	Oct15

		0, 40.0: 50 100 150 200 250 300 350 
		0, 30.0: 50 100 150 200 250 300 350 
		0, 20.0: 50 100 150 200 250